In [2]:
# Installing libraries
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall
!pip install spacy
!pip install textblob
!python -m spacy download en_core_web_sm

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /private/var/folders/62/nrb2x8tn6nl1qtqn66_155fc0000gn/T/pip-req-build-52cdfsva
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /private/var/folders/62/nrb2x8tn6nl1qtqn66_155fc0000gn/T/pip-req-build-52cdfsva
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 9ca68d37027af9f6a30d54640347ce3b2e2694b3
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7620 sha256=277524f0c90edabbe99b2e67439b9049113a4537d1d0edd5468d44f019a50090
  Stored in directory: /private/var/folders/62/nrb2x8tn6nl1qtqn66_155fc0000gn/T/pip-ephem-wheel-cache-32dq70fn/wheels/d1/c3/bb/559fe93e652b51cbc532f17e9693f3b70055f8560cf06c1fb3
Successfully built preprocess-kgptalkie


In [38]:
# Libraries
import dotenv
import os
from zipfile import ZipFile
import pandas as pd
import seaborn as sns
import preprocess_kgptalkie as ps
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
# Importing credentials
envVar = dotenv.dotenv_values('.env')
print(envVar.keys())

odict_keys(['usr', 'pwd'])


In [34]:
# Setting kaggle credentials and downloading dataset
os.environ['KAGGLE_USERNAME'] = envVar['usr']
os.environ['KAGGLE_KEY'] = envVar['pwd']
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 97%|████████████████████████████████████▉ | 25.0M/25.7M [00:01<00:00, 20.0MB/s]
100%|██████████████████████████████████████| 25.7M/25.7M [00:01<00:00, 15.6MB/s]


In [35]:
# Unzipping data
path = 'imdb-dataset-of-50k-movie-reviews.zip'
with ZipFile(path, 'r') as zip:
    zip.extractall()
    print('done')

done


In [36]:
# Reading data as df
df = pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
